In [2]:
apsimx_wrapper_options <- function(apsimx_path,
                                   apsimx_file, ... ) {

  # Template list
  options <- list()
  options$apsimx_path <- character(0)
  options$apsimx_file <- character(0)
  options$time_display <- FALSE
  options$warning_display <- TRUE
  options$multi_process <- FALSE
  options$predicted_table_name <- 'Report'
  options$observed_table_name <- 'Observed'
  options$variable_names <- character(0)
  options$met_files_path <- character(0)
  options$obs_files_path <- character(0)

  # For getting the template
  # running apsimx_wrapper_options
  if (! nargs()) return(options)

  # For fixing mandatory fields values
  options$apsimx_path <- apsimx_path
  options$apsimx_file <- apsimx_file

  # Fixing optional fields,
  # if corresponding to exact field names
  # in options list
  list_names <- names (options)
  add_args <- list(...)

  for (n in names(add_args)) {
    if ( n %in% list_names) {
      options[[n]] <- add_args[[n]]
    }
  }

  # Adding dates variable for getting them in outputs
  if ( ! is.element("Clock.Today", options$variable_names ) ) {
    if (length(options$variable_names) > 0 && grepl('Predicted.', options$variable_names[1])) {
      options$variable_names <- c(options$variable_names, "Predicted.Clock.Today")
    } else {
      options$variable_names <- c(options$variable_names, "Clock.Today")
    }
  }

  return(options)
}

In [ ]:
model_options <- apsimx_wrapper_options(
  apsimx_path = "/usr/local/bin/Models",
  apsimx_file = file.path(getwd(), "template.apsimx"),
  variable_names = c("Wheat.Leaf.LAI"),
  predicted_table_name = "DailyReport",
  observed_table_name = "Observed",
  met_files_path = getwd(),
  obs_files_path = getwd()
)

apsimx_wrapper <- function(model_options, param_values = NULL) {
  # print(model_options)
  # print(param_values)
  print("Rodando!!")

  return(list(
    biomass = 20,
    yield = 100
    # sim_list
    # db_file_name
  ))
}

sim_before_optim <- apsimx_wrapper(model_options = model_options)
sim_before_optim

sim_par_forcing <- apsimx_wrapper(
  param_values = c(
    .Simulations.Replacements.Wheat.Leaf.ExtinctionCoeff.VegetativePhase.FixedValue=0.4,
    .Simulations.Replacements.Wheat.Leaf.Photosynthesis.RUE.FixedValue=1.4
  ),
  model_options = model_options
)
sim_par_forcing

# obs_list <- read_apsimx_output(
#   sim_before_optim$db_file_name,
#   model_options$observed_table_name,
#   model_options$variable_names,
#   names(sim_before_optim$sim_list)
# )
# obs_list <- filter_obs(
#   obs_list,
#   situation = "GattonRowSpacingRowSpace25cm",
#   include = TRUE
# )

$apsimx_path
[1] "/usr/local/bin/Models"

$apsimx_file
[1] "/home/luanabeckerdaluz/git/luanabeckerdaluz/rapsimx/examples/template.apsimx"

$time_display
[1] FALSE

$warning_display
[1] TRUE

$multi_process
[1] FALSE

$predicted_table_name
[1] "DailyReport"

$observed_table_name
[1] "Observed"

$variable_names
[1] "Wheat.Leaf.LAI" "Clock.Today"   

$met_files_path
[1] "/home/luanabeckerdaluz/git/luanabeckerdaluz/rapsimx/examples"

$obs_files_path
[1] "/home/luanabeckerdaluz/git/luanabeckerdaluz/rapsimx/examples"

NULL


$biomass
[1] 20

$yield
[1] 100

$apsimx_path
[1] "/usr/local/bin/Models"

$apsimx_file
[1] "/home/luanabeckerdaluz/git/luanabeckerdaluz/rapsimx/examples/template.apsimx"

$time_display
[1] FALSE

$warning_display
[1] TRUE

$multi_process
[1] FALSE

$predicted_table_name
[1] "DailyReport"

$observed_table_name
[1] "Observed"

$variable_names
[1] "Wheat.Leaf.LAI" "Clock.Today"   

$met_files_path
[1] "/home/luanabeckerdaluz/git/luanabeckerdaluz/rapsimx/examples"

$obs_files_path
[1] "/home/luanabeckerdaluz/git/luanabeckerdaluz/rapsimx/examples"

.Simulations.Replacements.Wheat.Leaf.ExtinctionCoeff.VegetativePhase.FixedValue 
                                                                            0.4 
             .Simulations.Replacements.Wheat.Leaf.Photosynthesis.RUE.FixedValue 
                                                                            1.4 


$biomass
[1] 20

$yield
[1] 100

In [ ]:
res <- estim_param(
  obs_list = obs_list,
  model_function = apsimx_wrapper,
  model_options = model_options,
  optim_options = list(
    nb_rep = 7,
    maxeval = 500,
    xtol_rel = 1e-03,
    out_dir = getwd(),
    ranseed = 1234
  ),
  param_info = list(
    lb = c(
      .Simulations.Replacements.Wheat.Leaf.ExtinctionCoeff.VegetativePhase.FixedValue = 0.4,
      .Simulations.Replacements.Wheat.Leaf.Photosynthesis.RUE.FixedValue = 1.4
    ),
    ub = c(
      .Simulations.Replacements.Wheat.Leaf.ExtinctionCoeff.VegetativePhase.FixedValue = 0.6,
      .Simulations.Replacements.Wheat.Leaf.Photosynthesis.RUE.FixedValue = 1.6
    )
  )
)

res$final_values

In [ ]:
sim_after_optim <- apsimx_wrapper(param_values = res$final_values,
                               model_options = model_options)

In [ ]:
sensi <- rapsimx::RApsimxSensitivity(
  folder = "tmp_sensi",
  copy_met_data_from = "../met_data/",
  multicores = 2,
  models_command = NULL
)

In [ ]:
sensi$import_or_overwrite_problem("problem.R")
sensi$problem

In [ ]:
sensi$generate_samples(method = "LHS", n_samples = 10L)

In [ ]:
head(sensi$samples_df)
sensi$samples_df.plot()

In [ ]:
sensi$info()

In [ ]:
sensi$generate_apsimxs(
  sensit_base_sim_filepath = "../base_simulations/Soy-BR-clean.apsimx",
  runs_only_some_n = 10L,
  dry_run = FALSE
)

In [ ]:
sensi$run(
  runs_only_some_n = 2L,
  simulations_names = NULL,
  ids_to_run = NULL,
  dry_run = FALSE
)

In [ ]:
sensi$summarize(
  # ids_to_summarize = c(1,10),
  # number_of_fields_to_check = 32L,
  # runs_only_some_n = 2L,
  # dry_run = FALSE
)

In [ ]:
head(sensi$summarize_df)
# sensi$info()